<a href="https://colab.research.google.com/github/okana2ki/gai4e/blob/main/ai_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

まず、↑上の**「ドライブにコピー」をクリック**して、このコラボ・ノートブックを自分のGoogleドライブにコピーします。コピーすることで、変更を保存したり、保存したノートブックを共有できるようになります。
スマートフォンの場合は、画面左上の「メニューアイコン」をクリック→「ファイル」をクリック→「ドライブにコピーを保存」をクリック。

コラボ初心者向け準備

- コラボ（あなたが今使っているこの環境）について知りたい人は、[ここ](https://colab.research.google.com/github/okana2ki/intro-to-DS/blob/main/Colab_intro.ipynb)を参照して下さい
- このコラボ環境内で、生成AIに手伝ってもらいたい人は、[ここ](https://colab.research.google.com/github/okana2ki/intro-to-DS/blob/main/bingo.ipynb)を参照して下さい

# 「AIエージェント」を作って動かして理解しよう（初学者向け）

## 解説

### AIエージェントとは

AIエージェントの定義はまだ定まっていませんが、ここでは、次の２つの特徴を持つエージェントを指すことにします。

1. 外部ツールが使える

2. 人がタスクだけ与えれば、自動で計画を立てて、その計画を自動で完了まで実行してくれる

### 外部ツールを使うしくみ

使えるツールの一覧を大規模言語モデル（LLM）に与えておき、「与えられたタスクに対して使うべきツールと、そのツールに渡すパラメータを出力するよう、大規模言語モデル（LLM）に依頼する」ことで実現

詳しくは、「[Gemini API を使用した関数呼び出し](https://ai.google.dev/gemini-api/docs/function-calling?hl=ja&example=meeting)」を参照

### 自律的な計画立案と実行のしくみ（一例）

ReActパターン：以下の1.から3.の繰り返し

1. Reasoning（推論）：推論履歴と観察履歴を踏まえて、次に何をすべきか（次の行動またはユーザへの最終応答）を考える（**ここをLLMで実行**）
2. Acting（行動）：ツールの選択と実行（推論の結果がツールの使用であれば、指定されたツールを呼び出す；ユーザへの最終応答であればそれを出力してループを抜ける）
3. Observing（観察）：ツール呼び出しの結果を観察履歴に追加

この他にもいろいろなパターンがあります。→たとえば、[ここ①](https://qiita.com/syukan3/items/5becdc4f786aedbfa80a)や[ここ②](https://qiita.com/syukan3/items/ddb51cd2e6b853f0f948)を参照

### Python SDKの[自動関数呼び出し](https://ai.google.dev/gemini-api/docs/function-calling?hl=ja&example=meeting#automatic_function_calling_python_only)

SDK は自動的に次の処理を行います。

1. LLMからの関数呼び出しレスポンスを検出します。
2. 対応する関数を自動で呼び出します。
3. 関数のレスポンスをLLMに送り返します。
4. LLMの最終的なテキスト・レスポンスを返します。

この自動関数呼び出し（SDKが自動でやってくれる）を使うと、「LLMの出力に応じて関数呼び出しをして、その結果をLLMに入力する」コードをあなたが書く必要がありません。この点で楽なので、コーディングの初心者向けですが、商品の購入などのように関数の実行が重大な影響を与えうるもののときは要注意です。

### 大規模言語モデル（LLM）の選定について

2025年10月6日時点での無料枠の[レート制限](https://ai.google.dev/gemini-api/docs/rate-limits?hl=ja)は以下の通り
```
モデル                RPM TPM       RPD
Gemini 2.5 Pro        5   250,000   100
Gemini 2.5 Flash      10  250,000   250
Gemini 2.5 Flash-Lite 15  250,000   1,000
```
- **2.5 Flash-Lite**のレート制限が、この中では一番ゆるく余裕があり、**自動関数呼び出しを使う場合**は、動作的にもFlash-Liteで問題がなさそう。
- ただし、一番ゆるいとは言え、自動関数呼び出しを使う場合は、自動で計画を立てるために1回LLMを使い、その後、計画を実行するとき、計画のステップ毎にLLMへのリクエストが発生するとした場合、**高速に自動でLLMへのリクエストが発生**することになるため、**わりとすぐ、レート制限にかかって**しまう可能性がある。
- 下記で**手動制御版**のエージェントを試しているが、この場合は、**2.5 Flash-Liteでは、安定した計画実行ができなかった。**
- より高性能の2.5 Flashモデルにすると、手動制御版でも安定動作したので、**手動制御版を使う場合は、2.5 Flashモデルを推奨**。
- 自動関数呼び出しは、がんばっていろいろチューニングしたり、安定しない場合の回復策を施したりているはずなので、2.5 Flash-Liteでも動作するのだと思う。

### 参考資料

- [Gemini API の Function Calling を試す](https://note.com/npaka/n/nee0f849fad5d)
- [Gemini API の Function Calling 機能で LLM Agent を実装する](https://zenn.dev/google_cloud_jp/articles/3fa4da1eb63948)

## APIキーの入手と設定

Gemini API を使うためにはAPIキーが必要です。

### 1. キーの入手

<a class="button" href="https://aistudio.google.com/app/apikey" target="_blank" rel="noopener noreferrer">Get an API key</a>

↑Gemini API の API key をまだ持っていない場合は、上のリンクをクリックして、キーを入手する

- 上のリンクをクリックすると、「APIキーを作成 (Get API key)」ボタンや、すでに持っているキーの一覧が表示される（保有しているキーを使う場合は、キーの一覧中のAPIキーをクリックするとコピーボタンが現れる）
- キーを新たに作る場合は、「APIキーを作成(Get API key)」ボタンをクリック

### 2. Colabのシークレットマネージャにキーを登録

- 左側のパネルの鍵アイコン🔑をクリックして現れたパネル内の「新しいシークレットを追加」をクリック
- スマホの人は鍵アイコン🔑が見つからないと思います；スマホの場合は、「PC版サイト」に切り替える必要があります（Androidの場合は[ここ](https://atmarkit.itmedia.co.jp/ait/articles/1412/15/news098.html)を／iPhone, iPadの場合は[ここ](https://atmarkit.itmedia.co.jp/ait/articles/1411/10/news130.html)を見てPC版サイトへの切り替え方を調べて下さい）
- 「名前」欄に GOOGLE_API_KEY と入力
- 「値」欄に、クリップボードにコピーしておいたたAPIキーを貼り付け
- 「ノートブックからのアクセス」がを on にする


## 準備

以下のコードセルを上から順に実行していって下さい。コードセルにマウスオンすると、セルの左端に再生ボタンが現れるので、それをクリックすると実行できます。

In [ ]:
!pip install plyer

In [ ]:
from google import genai
from google.genai import types
import datetime
import json
import os
from plyer import notification
import time
from google.colab import userdata

## ツール定義

In [ ]:
# ============================================
# ツール定義
# ============================================

def get_current_time() -> dict:
    """現在時刻を取得します"""
    now = datetime.datetime.now()
    result = {
        "date": now.strftime("%Y年%m月%d日"),
        "time": now.strftime("%H:%M:%S"),
        "weekday": ["月", "火", "水", "木", "金", "土", "日"][now.weekday()] + "曜日",
        "hour": now.hour
    }
    print(f"   📅 現在時刻: {result['date']} {result['time']} ({result['weekday']})")
    return result


def save_file(filename: str, content: str) -> dict:
    """ファイルに内容を保存します"""
    try:
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(content)
        print(f"   💾 ファイル保存: {filename}")
        return {"status": "成功", "filename": filename, "size": len(content)}
    except Exception as e:
        return {"status": "失敗", "error": str(e)}


def read_file(filename: str) -> dict:
    """ファイルから内容を読み込みます"""
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            content = f.read()
        print(f"   📖 ファイル読込: {filename} ({len(content)}文字)")
        return {"status": "成功", "content": content}
    except Exception as e:
        return {"status": "失敗", "error": str(e)}


def calculate(expression: str) -> dict:
    """数式を計算します"""
    try:
        result = eval(expression)
        print(f"   🔢 計算: {expression} = {result}")
        return {"expression": expression, "result": result}
    except Exception as e:
        return {"error": str(e)}


def send_notification(title: str, message: str) -> dict:
    """デスクトップ通知を送信します"""
    try:
        notification.notify(
            title=title,
            message=message,
            app_name='自律AIエージェント',
            timeout=5
        )
        print(f"   🔔 通知送信: {title} - {message}")
        return {"status": "成功", "title": title}
    except Exception as e:
        # 通知が使えない環境でもエラーにしない
        print(f"   📢 通知（コンソール）: {title} - {message}")
        return {"status": "成功（コンソール出力）", "title": title}


def add_todo(task: str, priority: str = "normal") -> dict:
    """ToDoリストにタスクを追加します"""
    try:
        if os.path.exists("todos.json"):
            with open("todos.json", 'r', encoding='utf-8') as f:
                todos = json.load(f)
        else:
            todos = []

        new_task = {
            "id": len(todos) + 1,
            "task": task,
            "priority": priority,
            "completed": False,
            "created_at": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
        todos.append(new_task)

        with open("todos.json", 'w', encoding='utf-8') as f:
            json.dump(todos, f, ensure_ascii=False, indent=2)

        print(f"   ✅ ToDo追加: {task} (優先度: {priority})")
        return {"status": "成功", "task": new_task}
    except Exception as e:
        return {"status": "失敗", "error": str(e)}


def get_todos() -> dict:
    """ToDoリストを取得します"""
    try:
        if os.path.exists("todos.json"):
            with open("todos.json", 'r', encoding='utf-8') as f:
                todos = json.load(f)
            incomplete = [t for t in todos if not t["completed"]]
            print(f"   📋 ToDo確認: {len(incomplete)}件の未完了タスク")
            return {"status": "成功", "todos": todos, "incomplete_count": len(incomplete)}
        else:
            print(f"   📋 ToDo確認: タスクなし")
            return {"status": "成功", "todos": [], "incomplete_count": 0}
    except Exception as e:
        return {"status": "失敗", "error": str(e)}


def analyze_numbers(numbers: str) -> dict:
    """数値リストを分析します（カンマ区切り）"""
    try:
        nums = [float(x.strip()) for x in numbers.split(',')]
        result = {
            "count": len(nums),
            "sum": sum(nums),
            "average": sum(nums) / len(nums),
            "max": max(nums),
            "min": min(nums)
        }
        print(f"   📊 数値分析: {len(nums)}個の数値を分析")
        return result
    except Exception as e:
        return {"error": str(e)}


## エージェント

### 完全自律型エージェント

LLMへのリクエスト回数がブラックボックスになっており、カウントできていない。おそらく会話履歴を見ると分かると思うので、後日、試みる。

In [ ]:
# ============================================
# 完全自律型エージェント
# ============================================

class AutonomousAgent:
    """
    完全自律型AIエージェント

    特徴：
    - タスクだけ与えればAIが計画を立てる
    - 自動で必要なツールを選択・実行
    - 結果を見て次の行動を自律的に決定
    - タスク完了まで自動でループ
    """

    def __init__(self, tools_functions: list, max_iterations: int = 10):
        """
        Args:
            tools_functions: 利用可能なツール（関数）のリスト
            max_iterations: 最大反復回数（無限ループ防止）
        """
        # Retrieve API key from Colab secrets
        api_key = userdata.get('GOOGLE_API_KEY')
        # Use api_key when initializing the client
        self.client = genai.Client(api_key=api_key)
        self.tools_map = {func.__name__: func for func in tools_functions}

        # 自動実行モード
        self.config = types.GenerateContentConfig(
            tools=tools_functions,
            # 自動実行を有効化（これが重要！）
            # automatic_function_calling は有効のままにする
        )

        self.max_iterations = max_iterations
        self.iteration_count = 0
        self.request_count = 0 # APIリクエストカウンター

    def run(self, task: str, verbose: bool = True) -> str:
        """
        タスクを完全自律的に実行

        Args:
            task: 実行するタスク（自然言語）
            verbose: 詳細表示するか

        Returns:
            最終的な実行結果
        """
        if verbose:
            print("="*70)
            print(f"🚀 自律型AIエージェント起動")
            print("="*70)
            print(f"📝 タスク: {task}")
            print("="*70)
            print()

        # タスクに対して、AIに計画立案を促すプロンプトを追加
        enhanced_task = f"""
以下のタスクを完了してください：

{task}

重要な指示：
1. まず、タスクを完了するために必要なステップを考えてください
2. 各ステップで適切なツールを選択して実行してください
3. ツールの実行結果を確認し、次に何をすべきか判断してください
4. すべてのステップが完了したら、最終報告をしてください
5. 途中で問題が発生したら、別のアプローチを試してください

利用可能なツール一覧:
{', '.join(self.tools_map.keys())}

それでは、タスクを開始してください。
"""

        # Gemini の automatic function calling を使用
        # SDKが自動的にツール呼び出しループを処理
        response = self.client.models.generate_content(
            # model="gemini-2.5-flash",
            model="gemini-2.5-flash-lite",  # モデルを Flash-Lite に変更（レート制限にかかりにくくするため）
            contents=enhanced_task,
            config=self.config
        )
        self.request_count += 1 # リクエストをカウント (自律動作開始のための最初の呼び出しのみ)

        if verbose:
            print()
            print("="*70)
            print("✅ タスク完了")
            print("="*70)
            print(f"📄 最終報告:\n{response.text}")
            print("="*70)
            # 自動関数呼び出しによる内部リクエストは、このカウントに含まれないことに注意
            print(f"📊 このタスクでのAPIリクエスト数 (自律動作開始のための最初の呼び出し): {self.request_count}")

        return response.text

### 対話履歴を表示する完全自律型エージェント

In [ ]:
# ============================================
# 対話履歴を表示する完全自律型エージェント
# ============================================

class AutonomousAgentWithHistory:
    """
    完全自律型AIエージェント（対話履歴表示機能付き）

    特徴：
    - タスクだけ与えればAIが計画を立てる
    - 自動で必要なツールを選択・実行
    - 結果を見て次の行動を自律的に決定
    - タスク完了まで自動でループ
    - 対話履歴（関数呼び出しを含む）を表示可能
    """

    def __init__(self, tools_functions: list, max_iterations: int = 10):
        """
        Args:
            tools_functions: 利用可能なツール（関数）のリスト
            max_iterations: 最大反復回数（無限ループ防止）
        """
        # Retrieve API key from Colab secrets
        api_key = userdata.get('GOOGLE_API_KEY')
        self.client = genai.Client(api_key=api_key)
        self.tools_map = {func.__name__: func for func in tools_functions}

        # 自動実行モード
        self.config = types.GenerateContentConfig(
            tools=tools_functions,
        )

        self.max_iterations = max_iterations
        self.request_count = 0
        self.history = []  # 対話履歴を保存

    def print_history(self):
        """対話履歴を整形して表示"""
        print("\n" + "="*70)
        print("📜 対話履歴（詳細）")
        print("="*70)

        for i, item in enumerate(self.history, 1):
            print(f"\n--- エントリ {i} ---")
            print(f"ロール: {item['role']}")

            if 'parts' in item:
                for j, part in enumerate(item['parts'], 1):
                    print(f"\n  パート {j}:")

                    # テキスト
                    if hasattr(part, 'text') and part.text:
                        print(f"    タイプ: テキスト")
                        print(f"    内容: {part.text[:200]}..." if len(part.text) > 200 else f"    内容: {part.text}")

                    # 関数呼び出し
                    elif hasattr(part, 'function_call') and part.function_call:
                        fc = part.function_call
                        print(f"    タイプ: 関数呼び出し")
                        print(f"    関数名: {fc.name}")
                        print(f"    引数: {dict(fc.args)}")

                    # 関数レスポンス
                    elif hasattr(part, 'function_response') and part.function_response:
                        fr = part.function_response
                        print(f"    タイプ: 関数レスポンス")
                        print(f"    関数名: {fr.name}")
                        print(f"    結果: {dict(fr.response)}")

        print("\n" + "="*70)

    def run(self, task: str, verbose: bool = True, show_history: bool = True) -> str:
        """
        タスクを完全自律的に実行

        Args:
            task: 実行するタスク（自然言語）
            verbose: 詳細表示するか
            show_history: 対話履歴を表示するか

        Returns:
            最終的な実行結果
        """
        if verbose:
            print("="*70)
            print(f"🚀 自律型AIエージェント起動（履歴記録機能付き）")
            print("="*70)
            print(f"📝 タスク: {task}")
            print("="*70)
            print()

        enhanced_task = f"""
以下のタスクを完了してください：

{task}

重要な指示：
1. まず、タスクを完了するために必要なステップを考えてください
2. 各ステップで適切なツールを選択して実行してください
3. ツールの実行結果を確認し、次に何をすべきか判断してください
4. すべてのステップが完了したら、最終報告をしてください
5. 途中で問題が発生したら、別のアプローチを試してください

利用可能なツール一覧:
{', '.join(self.tools_map.keys())}

それでは、タスクを開始してください。
"""

        # チャットセッションを使用（履歴が自動的に管理される）
        chat = self.client.chats.create(
            model="gemini-2.5-flash-lite",
            config=self.config
        )

        response = chat.send_message(enhanced_task)
        self.request_count += 1

        # 履歴を取得して保存
        self.history = []
        for message in chat.history:
            self.history.append({
                'role': message.role,
                'parts': message.parts
            })

        if verbose:
            print()
            print("="*70)
            print("✅ タスク完了")
            print("="*70)
            print(f"📄 最終報告:\n{response.text}")
            print("="*70)
            print(f"📊 チャット履歴のメッセージ数: {len(self.history)}")

        # 対話履歴を表示
        if show_history:
            self.print_history()

        return response.text


### 手動制御版エージェント

In [ ]:
# ===================================================
# 手動制御版エージェント（学習用：内部の動きが見える）
# ===================================================

class ManualAutonomousAgent:
    """
    手動制御版の自律型エージェント

    automatic_function_calling を無効化して、
    AIの思考過程とツール選択の様子を可視化
    """

    def __init__(self, tools_functions: list, max_iterations: int = 10):
        # Retrieve API key from Colab secrets
        api_key = userdata.get('GOOGLE_API_KEY')
        # Use api_key when initializing the client
        self.client = genai.Client(api_key=api_key)
        self.tools_map = {func.__name__: func for func in tools_functions}

        # 手動制御モード
        self.config = types.GenerateContentConfig(
            tools=tools_functions,
            automatic_function_calling=types.AutomaticFunctionCallingConfig(
                disable=True  # 手動制御で内部を見る
            )
        )

        self.max_iterations = max_iterations
        self.request_count = 0 # APIリクエストカウンター


    def run(self, task: str) -> str:
        """タスクを実行（手動制御版）"""
        print("="*70)
        print(f"🔍 手動制御版エージェント起動（内部の動きが見える）")
        print("="*70)
        print(f"📝 タスク: {task}")
        print("="*70)
        print()

        # 強化されたプロンプト
        enhanced_task = f"""
以下のタスクを段階的に完了してください：

{task}

タスク実行の指針：
1. 最初に、タスク達成のための計画を立ててください。
2. 計画の各ステップを実行するために、**ツール呼び出しの形式で**必要なアクションを指定してください。AIモデルは、ツール呼び出しの形式で応答を生成する必要があります。
3. ツールの実行結果を受け取ったら、次のステップに進むか、タスク完了を判断してください。
4. 全てのステップが完了したら「タスク完了」と明示してください。

利用可能なツール: {', '.join(self.tools_map.keys())}
"""

        contents = [enhanced_task]

        for iteration in range(self.max_iterations):
            print(f"\n{'='*70}")
            print(f"🔄 ステップ {iteration + 1}")
            print(f"{'='*70}")

            # AIにメッセージ送信
            response = self.client.models.generate_content(
                model="gemini-2.5-flash",  # 手動制御版では、このモデルを推奨
                # model="gemini-2.5-flash-lite",  # モデルを Flash-Lite に変更するとツール呼び出しが不安定
                contents=contents,
                config=self.config
            )
            self.request_count += 1 # リクエストをカウント


            candidate = response.candidates[0]
            part = candidate.content.parts[0]

            # テキスト応答の場合
            if hasattr(part, 'text') and part.text:
                print(f"\n💭 AIの思考・報告:")
                print(f"   {part.text}")

                # タスク完了を示すキーワードチェック
                if any(keyword in part.text for keyword in ["タスク完了", "完了しました", "全て終わりました"]):
                    print(f"\n{'='*70}")
                    print("✅ タスク完了を検出")
                    print(f"{'='*70}")
                    print(f"📊 このタスクでのAPIリクエスト数: {self.request_count}") # リクエスト数を表示
                    return part.text

                # 最後の反復の場合は終了
                if iteration == self.max_iterations - 1:
                    print(f"\n⚠️ 最大反復回数に到達しました。")
                    print(f"📊 このタスクでのAPIリクエスト数: {self.request_count}") # リクエスト数を表示
                    return part.text

            # ツール呼び出しの場合
            elif hasattr(part, 'function_call') and part.function_call:
                function_call = part.function_call
                function_name = function_call.name
                function_args = dict(function_call.args)

                print(f"\n🤖 AIの判断:")
                print(f"   ツール: {function_name}")
                print(f"   引数: {function_args}")
                print(f"\n⚙️  実行中...")

                # ツール実行
                if function_name in self.tools_map:
                    try:
                        result = self.tools_map[function_name](**function_args)

                        print(f"\n✨ 実行結果:")
                        print(f"   {result}")

                        # 会話履歴に追加 (AIの関数呼び出しとツール実行結果)
                        contents.append(candidate.content) # AIの関数呼び出しパートを追加
                        function_response_part = types.Part.from_function_response(
                            name=function_name,
                            response={"result": result}
                        )
                        contents.append(
                            types.Content(
                                role="user",
                                parts=[function_response_part]
                            )
                        )
                    except Exception as e:
                        error_msg = f"ツール実行エラー: {e}"
                        print(f"\n❌ 実行エラー:")
                        print(f"   {error_msg}")
                        # エラーを会話履歴に追加してAIにフィードバック
                        contents.append(candidate.content) # AIの関数呼び出しパートを追加
                        error_part = types.Part.from_function_response(
                             name=function_name,
                             response={"error": error_msg}
                        )
                        contents.append(
                            types.Content(
                                role="user",
                                parts=[error_part]
                            )
                        )


                else:
                    print(f"\n❌ エラー: {function_name} は存在しません")
                    print(f"📊 このタスクでのAPIリクエスト数: {self.request_count}") # リクエスト数を表示
                    return f"エラー: {function_name} は存在しません"
            else:
                print(f"\n🤔 予期しない応答形式:")
                print(f"   {part}")
                print(f"📊 このタスクでのAPIリクエスト数: {self.request_count}") # リクエスト数を表示
                return "予期しない応答形式"

        print(f"\n⚠️ 最大反復回数に到達しました。")
        print(f"📊 このタスクでのAPIリクエスト数: {self.request_count}") # リクエスト数を表示
        return "最大反復回数に到達しました"

## タスク例

↓下記のコードセルの最後の行の `main(6)`の括弧の中の半角数字を1から6のいずれかに書き換えて実行して下さい。指定した数字に応じて1番から6番までのタスク例のどれか1つが実行されます。

In [ ]:
# ============================================
# 例題集（難易度順）
# ============================================

def main(task_number: int = None):
    # 利用可能なツール
    tools = [
        get_current_time,
        save_file,
        read_file,
        calculate,
        send_notification,
        add_todo,
        get_todos,
        analyze_numbers
    ]

    print("\n" + "🎓 自律型AIエージェント - 例題デモ\n")

    # agent = AutonomousAgent(tools)
    agent = AutonomousAgentWithHistory(tools)  # 対話履歴表示
    manual_agent = ManualAutonomousAgent(tools, max_iterations=8)

    tasks = {
        1: ("基本的な自律実行（簡単）", """
現在時刻を取得して、その情報をtime_report.txtというファイルに保存してください。
"""),
        2: ("複数ステップの自律的計画実行（中級）", """
以下のタスクを順番に実行してください：
1. 「10, 20, 30, 40, 50」という数値リストを分析する
2. 分析結果（平均値、合計、最大値など）をanalysis_report.txtに保存する
3. 「データ分析が完了しました」という通知を送る
"""),
        3: ("条件分岐を含む自律実行（中級）", """
現在時刻を確認して：
- もし午前中（12時より前）なら「おはようございます」という通知を送る
- もし午後なら「こんにちは」という通知を送る
- そして、その判断理由をgreetings.txtに保存する
"""),
        4: ("データ処理パイプラインの自律構築（上級）", """
以下の一連の作業を自律的に完了してください：

1. 「100, 200, 150, 300, 250」という売上データを分析
2. 平均売上を計算
3. 平均売上が200以上なら「目標達成」、未満なら「未達成」と判定
4. 判定結果と分析データをsales_report.txtに整形して保存
5. 「売上分析完了：[判定結果]」という内容で通知を送る
"""),
        5: ("エラーリカバリーと代替手段の自律選択（上級）", """
以下のタスクを実行してください：

1. nonexistent_file.txtを読み込む
2. もし読み込みに失敗したら、代わりに「ファイルが見つかりませんでした」という内容で新しくerror_log.txtを作成
3. 最終的な状況を通知で報告
"""),
        6: ("手動制御版（内部の動きを見る学習用）", """
ToDoリストに以下のタスクを追加してください：
1. 「レポートを書く」（優先度: high）
2. 「買い物に行く」（優先度: normal）

その後、ToDoリストを確認して、何件のタスクがあるか報告してください。
""")
    }

    if task_number is None:
        print("実行したい例題の番号を指定してください (例: main(1))")
        for num, (title, _) in tasks.items():
            print(f"{num}: {title}")
    elif task_number in tasks:
        title, task_description = tasks[task_number]
        print(f"\n\n" + "📌 例題{}: {}".format(task_number, title))
        print("-" * 70)

        if task_number == 6:
            manual_agent.run(task_description)
        else:
            # agent.run(task_description)
            agent.run(  # 対話履歴表示対応
                task_description,
                verbose=True,       # 実行過程を表示
                show_history=True   # 対話履歴を表示
            )

    else:
        print(f"エラー: 例題番号 {task_number} は存在しません。")


if __name__ == "__main__":
    # 実行したい例題の番号を main() の引数に指定してください。
    # 例: main(1) で例題1を実行
    main(1)


🎓 自律型AIエージェント - 例題デモ



📌 例題6: 手動制御版（内部の動きを見る学習用）
----------------------------------------------------------------------
🔍 手動制御版エージェント起動（内部の動きが見える）
📝 タスク: 
ToDoリストに以下のタスクを追加してください：
1. 「レポートを書く」（優先度: high）
2. 「買い物に行く」（優先度: normal）

その後、ToDoリストを確認して、何件のタスクがあるか報告してください。



🔄 ステップ 1

💭 AIの思考・報告:
   計画：
1. タスク「レポートを書く」を優先度「high」でToDoリストに追加します。
2. タスク「買い物に行く」を優先度「normal」でToDoリストに追加します。
3. ToDoリストの内容を取得します。
4. 取得したToDoリストのタスク数を数え、報告します。



🔄 ステップ 2

🤖 AIの判断:
   ツール: add_todo
   引数: {'task': 'レポートを書く', 'priority': 'high'}

⚙️  実行中...
   ✅ ToDo追加: レポートを書く (優先度: high)

✨ 実行結果:
   {'status': '成功', 'task': {'id': 1, 'task': 'レポートを書く', 'priority': 'high', 'completed': False, 'created_at': '2025-10-05 06:40:19'}}

🔄 ステップ 3

🤖 AIの判断:
   ツール: add_todo
   引数: {'task': '買い物に行く', 'priority': 'normal'}

⚙️  実行中...
   ✅ ToDo追加: 買い物に行く (優先度: normal)

✨ 実行結果:
   {'status': '成功', 'task': {'id': 2, 'task': '買い物に行く', 'priority': 'normal', 'completed': False, 'created_at': '2025-10-05 06:40:19'}}

🔄 ステップ 4

🤖 A

# 演習

ツールの追加、エージェントの推論パターンの追加、タスク例の追加、など、いろいろやってみて欲しい。必要なヒントや材料を後日、加筆します。